0. Install and Import Dependencies 

In [1]:
!pip install mediapipe opencv-python pandas scikit-learn

In [1]:
import mediapipe as mp
import cv2

In [2]:
mp_drawing = mp.solutions.drawing_utils
mp_holistic = mp.solutions.holistic

1. Making some detections

In [3]:
cap = cv2.VideoCapture(0)
# Initiate holistic model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor Feed
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False        
        
        # Make Detections
        results = holistic.process(image)
        # print(results.face_landmarks)
        
        # face_landmarks, pose_landmarks, left_hand_landmarks, right_hand_landmarks
        
        # Recolor image back to BGR for rendering
        image.flags.writeable = True   
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # 1. Draw face landmarks
        mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS, 
                                 mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
                                 mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                                 )
        
        # 2. Right hand
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                                 )

        # 3. Left Hand
        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                                 )

        # 4. Pose Detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                 )
                        
        cv2.imshow('Raw Webcam Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

In [4]:
# results
# mediapipe.python.solution_base.SolutionOutputs
# results.pose_landmarks or results.pose_landmarks
# results.face_landmarks
# results.face_landmarks.landmark[0]
# results.face_landmarks.landmark[0].x
results.pose_landmarks.landmark[0].visibility
# 0 to 1 for pose_landmark 0 for pose_landmark as default

0.9999769330024719

2. Capture Landmarks and export to CSV

In [5]:
import csv
import os
import numpy as np

In [6]:
# Lets check how many total landmarks we have for structuring our csv file column name
# face model doesnt have a visibility coordinate so for those x,y,z will be 0
# Since we have to loop through all of there coordinates
num_coords = len(results.face_landmarks.landmark)+len(results.pose_landmarks.landmark)
num_coords

501

In [7]:
# Every class will have 4 colums x,y,z,v
# Creating header row
landmarks = ['class']
for val in range(1, num_coords+1):
    landmarks += ['x{}'.format(val), 'y{}'.format(val), 'z{}'.format(val), 'v{}'.format(val)]
# landmarks

In [8]:
# export our colums into csv
with open('coords.csv', mode='w',newline='') as f:
    csv_writer = csv.writer(f, delimiter=',', quotechar='"',quoting=csv.QUOTE_MINIMAL)
    csv_writer.writerow(landmarks)

In [9]:
class_name = 'Shame'

In [2]:
cap = cv2.VideoCapture(0)
# Initiate holistic modelq
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    while cap.isOpened():
        ret, frame = cap.read()
# When we read our image through open-cv but default the feed we get back is in BGR format
# But in order to work with media pipe we need our frames to be in RGB format
# So we recolor our feed        
        # Recolor Feed
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    # For peformance improvement we set writable flags to false
    # to prevent copying the image data but you're also then able to use the same image for rendering
        image.flags.writeable = False        
    # finally we use holistic model form the with line, using the process method to actually make detections
    # in which we pass our image variable    
        # Make Detections
        results = holistic.process(image)
        # print(results.face_landmarks)
        
        # face_landmarks, pose_landmarks, left_hand_landmarks, right_hand_landmarks
        
     # flatten() collapses an array into single method   # Recolor image back to BGR for rendering
        image.flags.writeable = True   
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # 1. Draw face landmarks
        mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS, 
                                 mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
                                 mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                                 )
        
        # 2. Right hand
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                                 )

        # 3. Left Hand
        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                                 )

        # 4. Pose Detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                 )
        # Export coordinates
        try:
            # Extract Pose landmarks
            pose = results.pose_landmarks.landmark
            pose_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in pose]).flatten())
            # flatten() collapses an array into single method
            # Extract Face landmarks
            face = results.face_landmarks.landmark
            face_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in face]).flatten())
            
            # Concate rows
            row = pose_row+face_row
            
            # Append class name 
            row.insert(0, class_name)
            # beacause len(row) should be equal to 1+501*4
            
            # Export to CSV
            with open('coords.csv', mode='a', newline='') as f:
                csv_writer = csv.writer(f, delimiter=',', quotechar='"',quoting=csv.QUOTE_MINIMAL)
                csv_writer.writerow(row) 
            
        except:
            pass
                        
        cv2.imshow('Raw Webcam Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

NameError: name 'cv2' is not defined

3. Train Custom Model Using Scikit Learn

3.1 Read in collected Data and Process

In [11]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [12]:
df = pd.read_csv('coords.csv')
df

,class,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z499,v499,x500,y500,z500,v500,x501,y501,z501,v501
0,Shame,0.502358,0.441293,-1.339921,0.999304,0.532351,0.387770,-1.341192,0.998296,0.550225,...,-0.030493,0.0,0.572027,0.378117,-0.022856,0.0,0.576473,0.371853,-0.023679,0.0
1,Shame,0.505257,0.439769,-1.555150,0.999316,0.534597,0.383218,-1.557810,0.998341,0.552347,...,-0.026304,0.0,0.563330,0.362944,-0.019638,0.0,0.567319,0.358701,-0.020557,0.0
2,Shame,0.505237,0.424400,-1.494115,0.999348,0.534518,0.363431,-1.473169,0.998444,0.552339,...,-0.024979,0.0,0.558163,0.343978,-0.019376,0.0,0.562030,0.340344,-0.020428,0.0
3,Shame,0.503754,0.416382,-1.471812,0.999396,0.532767,0.354447,-1.457555,0.998559,0.551411,...,-0.023855,0.0,0.556130,0.337548,-0.017872,0.0,0.560054,0.334160,-0.018830,0.0
4,Shame,0.503935,0.400550,-1.436942,0.999415,0.532853,0.341885,-1.413850,0.998614,0.551610,...,-0.021761,0.0,0.555443,0.327744,-0.015324,0.0,0.559346,0.324323,-0.016178,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
449,Shame,0.510428,0.411097,-1.326465,0.999730,0.534332,0.343856,-1.301794,0.999404,0.550413,...,-0.023472,0.0,0.559468,0.344057,-0.013790,0.0,0.562972,0.340263,-0.014446,0.0
450,Shame,0.513084,0.412501,-1.372855,0.999736,0.535929,0.345782,-1.347772,0.999423,0.551530,...,-0.023471,0.0,0.561401,0.346350,-0.013233,0.0,0.564962,0.342360,-0.013804,0.0
451,Shame,0.515503,0.416952,-1.321785,0.999739,0.537703,0.349566,-1.293509,0.999430,0.553212,...,-0.022901,0.0,0.569302,0.348909,-0.009801,0.0,0.572696,0.344718,-0.010061,0.0
452,Shame,0.521226,0.421187,-1.390269,0.999742,0.542575,0.353217,-1.365260,0.999440,0.557664,...,-0.022872,0.0,0.572059,0.347260,-0.008658,0.0,0.575369,0.343198,-0.008820,0.0


In [77]:
df.head()

,class,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z499,v499,x500,y500,z500,v500,x501,y501,z501,v501
0,Happy,0.408155,0.455140,-1.114508,0.999988,0.440482,0.395876,-1.053173,0.999968,0.462196,...,-0.005582,0.0,0.479910,0.411320,0.015118,0.0,0.485391,0.407875,0.015641,0.0
1,Happy,0.406587,0.460972,-1.284935,0.999987,0.439867,0.398056,-1.221887,0.999967,0.461192,...,-0.005908,0.0,0.482900,0.411716,0.016200,0.0,0.488038,0.408916,0.016676,0.0
2,Happy,0.406345,0.466287,-1.288482,0.999987,0.439887,0.400790,-1.228090,0.999965,0.461138,...,-0.006527,0.0,0.484520,0.412045,0.014901,0.0,0.489724,0.409330,0.015302,0.0
3,Happy,0.406347,0.469537,-1.285370,0.999986,0.440093,0.402546,-1.223940,0.999964,0.461148,...,-0.007231,0.0,0.486172,0.412072,0.013964,0.0,0.491356,0.409385,0.014333,0.0
4,Happy,0.406640,0.472296,-1.303282,0.999984,0.440892,0.404554,-1.242484,0.999959,0.461598,...,-0.006833,0.0,0.486978,0.411651,0.014495,0.0,0.492145,0.409101,0.014877,0.0


In [78]:
df[df['class']=='Sad']

,class,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z499,v499,x500,y500,z500,v500,x501,y501,z501,v501
245,Sad,0.593950,0.274270,-0.850859,0.999984,0.614801,0.220343,-0.787649,0.999973,0.632130,...,0.012879,0.0,0.635064,0.230944,0.035277,0.0,0.639996,0.227230,0.036450,0.0
246,Sad,0.588077,0.272769,-1.051569,0.999983,0.611667,0.219464,-0.970072,0.999971,0.628694,...,0.013053,0.0,0.638812,0.231627,0.035363,0.0,0.643952,0.227536,0.036519,0.0
247,Sad,0.584866,0.271521,-1.055822,0.999982,0.610284,0.218346,-0.972662,0.999971,0.627148,...,0.012707,0.0,0.639000,0.231364,0.035052,0.0,0.644122,0.227486,0.036216,0.0
248,Sad,0.582863,0.270646,-1.039276,0.999978,0.609040,0.217569,-0.954949,0.999964,0.625896,...,0.012502,0.0,0.639069,0.231929,0.034710,0.0,0.644205,0.227942,0.035882,0.0
249,Sad,0.581238,0.271725,-0.930145,0.999901,0.608344,0.218369,-0.838438,0.999874,0.625061,...,0.013020,0.0,0.638053,0.234644,0.035448,0.0,0.643171,0.230490,0.036670,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
415,Sad,0.597533,0.377630,-1.057490,0.999259,0.623534,0.298681,-0.985050,0.998592,0.640608,...,-0.008905,0.0,0.661192,0.308611,0.012127,0.0,0.666265,0.304666,0.012449,0.0
416,Sad,0.599688,0.376968,-0.907715,0.999122,0.625231,0.298530,-0.842678,0.998403,0.642005,...,-0.007736,0.0,0.661517,0.307330,0.013427,0.0,0.666650,0.303426,0.013778,0.0
417,Sad,0.599501,0.369340,-0.923026,0.999128,0.625083,0.291658,-0.846949,0.998447,0.641925,...,-0.005569,0.0,0.659964,0.298539,0.015989,0.0,0.664945,0.295299,0.016433,0.0
418,Sad,0.599569,0.364377,-0.908901,0.999125,0.625715,0.287228,-0.829986,0.998468,0.642504,...,-0.005068,0.0,0.659089,0.294169,0.015724,0.0,0.664191,0.290657,0.016152,0.0


In [89]:
X = df.drop ('class',axis=1) #features
y = df['class'] #target value

In [90]:
y

0           Happy
1           Happy
2           Happy
3           Happy
4           Happy
          ...    
634    Victorious
635    Victorious
636    Victorious
637    Victorious
638    Victorious
Name: class, Length: 639, dtype: object

In [92]:
# creating train and test partition
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3, random_state=1234)
# X_train

,x1,y1,z1,v1,x2,y2,z2,v2,x3,y3,...,z499,v499,x500,y500,z500,v500,x501,y501,z501,v501
479,0.733405,0.382930,-0.444695,0.999997,0.743121,0.340814,-0.346208,0.999994,0.751843,0.342970,...,0.008689,0.0,0.755006,0.332226,0.037034,0.0,0.758344,0.330120,0.038734,0.0
214,0.387119,0.457535,-1.271406,0.998814,0.414828,0.394717,-1.211751,0.998880,0.433695,0.395640,...,-0.005197,0.0,0.455661,0.395949,0.017227,0.0,0.460093,0.392819,0.017900,0.0
7,0.409105,0.475694,-1.348590,0.999982,0.443617,0.407505,-1.288817,0.999955,0.463424,0.411316,...,-0.006046,0.0,0.489035,0.412515,0.015592,0.0,0.494242,0.409602,0.016049,0.0
435,0.569746,0.364733,-0.511626,0.999985,0.587763,0.321959,-0.449808,0.999970,0.600222,0.323158,...,0.005257,0.0,0.612693,0.329897,0.020376,0.0,0.616760,0.327601,0.021005,0.0
563,0.540497,0.326612,-0.610051,0.999952,0.564480,0.283003,-0.574988,0.999892,0.579573,0.284087,...,0.002145,0.0,0.586169,0.285610,0.016674,0.0,0.589873,0.283555,0.017260,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
279,0.587668,0.293168,-1.209590,0.999783,0.613561,0.232648,-1.132905,0.999546,0.632439,0.233535,...,0.013412,0.0,0.645478,0.245948,0.036548,0.0,0.651289,0.241705,0.037640,0.0
372,0.569767,0.378196,-1.159517,0.999561,0.605155,0.305003,-1.107067,0.999176,0.629042,0.303843,...,-0.013769,0.0,0.633751,0.313281,-0.006416,0.0,0.639783,0.308375,-0.007233,0.0
204,0.303650,0.458035,-1.023962,0.998928,0.338812,0.396007,-0.997168,0.999022,0.359346,0.397346,...,-0.014400,0.0,0.364920,0.401682,-0.030972,0.0,0.371633,0.395759,-0.033482,0.0
53,0.372134,0.484969,-1.030092,0.999944,0.402521,0.417826,-0.980451,0.999838,0.426442,0.418365,...,-0.012367,0.0,0.457869,0.422964,0.003645,0.0,0.463087,0.419769,0.003554,0.0


3.2 Train Machine Learning Classification Model

In [95]:
# We will use X_train or y_train values to fit our model i.e train our model
# and we will use X_test and y_test to validate it
from sklearn.pipeline import make_pipeline # for making a machine learning pipeline
from sklearn.preprocessing import StandardScaler # to our pipeline we pass through a preprocessing function called standard scaler
# levraging standart scalar STANDARDIZES the data, subtracts the main and divides it by the standard deviation
# to keep the data on a same plane so that no ideal feature overshadows the other feature
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
# Different models to train each and every one of them and get the best model fit for us

In [96]:
# we creating a dictionary
pipelines = {
    'lr': make_pipeline(StandardScaler(), LogisticRegression()),
    'rc': make_pipeline(StandardScaler(), RidgeClassifier()),
    'rf': make_pipeline(StandardScaler(), RandomForestClassifier()),
    'gb': make_pipeline(StandardScaler(), GradientBoostingClassifier()),
}

In [99]:
pipelines.values()

dict_values([Pipeline(steps=[('standardscaler', StandardScaler()),
                ('logisticregression', LogisticRegression())]), Pipeline(steps=[('standardscaler', StandardScaler()),
                ('ridgeclassifier', RidgeClassifier())]), Pipeline(steps=[('standardscaler', StandardScaler()),
                ('randomforestclassifier', RandomForestClassifier())]), Pipeline(steps=[('standardscaler', StandardScaler()),
                ('gradientboostingclassifier', GradientBoostingClassifier())])])

In [100]:
pipelines.keys()

dict_keys(['lr', 'rc', 'rf', 'gb'])

In [102]:
list(pipelines.values())

[Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('logisticregression', LogisticRegression())]),
 Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('ridgeclassifier', RidgeClassifier())]),
 Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('randomforestclassifier', RandomForestClassifier())]),
 Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('gradientboostingclassifier', GradientBoostingClassifier())])]

In [103]:
# Here we'll train the model
# Firstly we create an empty dictionary
# Then We try and loop through each one of these pipelines and train
fit_models = {}
for algo, pipeline in pipelines.items():
    model = pipeline.fit(X_train, y_train)
    fit_models[algo] = model

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [105]:
fit_models

{'lr': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('logisticregression', LogisticRegression())]),
 'rc': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('ridgeclassifier', RidgeClassifier())]),
 'rf': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('randomforestclassifier', RandomForestClassifier())]),
 'gb': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('gradientboostingclassifier', GradientBoostingClassifier())])}

In [106]:
fit_models['rf'].predict(X_test)

array(['Sad', 'Victorious', 'Sad', 'Victorious', 'Sad', 'Sad', 'Happy',
       'Sad', 'Happy', 'Happy', 'Victorious', 'Sad', 'Sad', 'Happy',
       'Happy', 'Happy', 'Happy', 'Sad', 'Happy', 'Victorious',
       'Victorious', 'Happy', 'Happy', 'Sad', 'Victorious', 'Happy',
       'Victorious', 'Sad', 'Victorious', 'Happy', 'Victorious',
       'Victorious', 'Sad', 'Happy', 'Happy', 'Happy', 'Victorious',
       'Sad', 'Happy', 'Sad', 'Happy', 'Victorious', 'Sad', 'Victorious',
       'Sad', 'Happy', 'Happy', 'Sad', 'Happy', 'Victorious',
       'Victorious', 'Victorious', 'Victorious', 'Victorious', 'Sad',
       'Sad', 'Victorious', 'Victorious', 'Happy', 'Happy', 'Sad',
       'Happy', 'Sad', 'Happy', 'Victorious', 'Sad', 'Happy', 'Happy',
       'Sad', 'Sad', 'Victorious', 'Victorious', 'Sad', 'Sad',
       'Victorious', 'Victorious', 'Happy', 'Sad', 'Sad', 'Sad', 'Happy',
       'Sad', 'Victorious', 'Happy', 'Happy', 'Happy', 'Happy',
       'Victorious', 'Happy', 'Victorious', 'Vi

In [110]:
fit_models['lr'].predict(X_test)

array(['Sad', 'Victorious', 'Sad', 'Victorious', 'Sad', 'Sad', 'Happy',
       'Sad', 'Happy', 'Happy', 'Victorious', 'Sad', 'Sad', 'Happy',
       'Happy', 'Happy', 'Happy', 'Sad', 'Happy', 'Victorious',
       'Victorious', 'Happy', 'Happy', 'Sad', 'Victorious', 'Happy',
       'Victorious', 'Sad', 'Victorious', 'Happy', 'Victorious',
       'Victorious', 'Sad', 'Happy', 'Happy', 'Happy', 'Victorious',
       'Sad', 'Happy', 'Sad', 'Happy', 'Victorious', 'Sad', 'Victorious',
       'Sad', 'Happy', 'Happy', 'Sad', 'Happy', 'Victorious',
       'Victorious', 'Victorious', 'Victorious', 'Victorious', 'Sad',
       'Sad', 'Victorious', 'Victorious', 'Happy', 'Happy', 'Sad',
       'Happy', 'Sad', 'Happy', 'Victorious', 'Sad', 'Happy', 'Happy',
       'Sad', 'Sad', 'Victorious', 'Victorious', 'Sad', 'Sad',
       'Victorious', 'Victorious', 'Happy', 'Sad', 'Sad', 'Sad', 'Happy',
       'Sad', 'Victorious', 'Happy', 'Happy', 'Happy', 'Happy',
       'Victorious', 'Happy', 'Victorious', 'Vi

In [112]:
fit_models['rc'].predict(X_test)

array(['Sad', 'Victorious', 'Sad', 'Victorious', 'Sad', 'Sad', 'Happy',
       'Sad', 'Happy', 'Happy', 'Victorious', 'Sad', 'Sad', 'Happy',
       'Happy', 'Happy', 'Happy', 'Sad', 'Happy', 'Victorious',
       'Victorious', 'Happy', 'Happy', 'Sad', 'Victorious', 'Happy',
       'Victorious', 'Sad', 'Victorious', 'Happy', 'Victorious',
       'Victorious', 'Sad', 'Happy', 'Happy', 'Happy', 'Victorious',
       'Sad', 'Happy', 'Sad', 'Happy', 'Victorious', 'Sad', 'Victorious',
       'Sad', 'Happy', 'Happy', 'Sad', 'Happy', 'Victorious',
       'Victorious', 'Victorious', 'Victorious', 'Victorious', 'Sad',
       'Sad', 'Victorious', 'Victorious', 'Happy', 'Happy', 'Sad',
       'Happy', 'Sad', 'Happy', 'Victorious', 'Sad', 'Happy', 'Happy',
       'Sad', 'Sad', 'Victorious', 'Victorious', 'Sad', 'Sad',
       'Victorious', 'Victorious', 'Happy', 'Sad', 'Sad', 'Sad', 'Happy',
       'Sad', 'Victorious', 'Happy', 'Happy', 'Happy', 'Happy',
       'Victorious', 'Happy', 'Victorious', 'Vi

In [115]:
fit_models['rc'].predict(X_test)

array(['Sad', 'Victorious', 'Sad', 'Victorious', 'Sad', 'Sad', 'Happy',
       'Sad', 'Happy', 'Happy', 'Victorious', 'Sad', 'Sad', 'Happy',
       'Happy', 'Happy', 'Happy', 'Sad', 'Happy', 'Victorious',
       'Victorious', 'Happy', 'Happy', 'Sad', 'Victorious', 'Happy',
       'Victorious', 'Sad', 'Victorious', 'Happy', 'Victorious',
       'Victorious', 'Sad', 'Happy', 'Happy', 'Happy', 'Victorious',
       'Sad', 'Happy', 'Sad', 'Happy', 'Victorious', 'Sad', 'Victorious',
       'Sad', 'Happy', 'Happy', 'Sad', 'Happy', 'Victorious',
       'Victorious', 'Victorious', 'Victorious', 'Victorious', 'Sad',
       'Sad', 'Victorious', 'Victorious', 'Happy', 'Happy', 'Sad',
       'Happy', 'Sad', 'Happy', 'Victorious', 'Sad', 'Happy', 'Happy',
       'Sad', 'Sad', 'Victorious', 'Victorious', 'Sad', 'Sad',
       'Victorious', 'Victorious', 'Happy', 'Sad', 'Sad', 'Sad', 'Happy',
       'Sad', 'Victorious', 'Happy', 'Happy', 'Happy', 'Happy',
       'Victorious', 'Happy', 'Victorious', 'Vi

3.3 Evaluate and Serialize Model

In [7]:
from sklearn.metrics import accuracy_score # to calculate performace of the models
import pickle # used by data scientist to save their trained model to disk
import cv2

In [118]:
for algo, model in fit_models.items():
    yhat = model.predict(X_test) #making a pridiction and storing it
    print(algo, accuracy_score(y_test, yhat)) # to workout how well the yhat model performed against y_test data

lr 1.0
rc 1.0
rf 1.0
gb 1.0


In [119]:
fit_models['rf'].predict(X_test)

array(['Sad', 'Victorious', 'Sad', 'Victorious', 'Sad', 'Sad', 'Happy',
       'Sad', 'Happy', 'Happy', 'Victorious', 'Sad', 'Sad', 'Happy',
       'Happy', 'Happy', 'Happy', 'Sad', 'Happy', 'Victorious',
       'Victorious', 'Happy', 'Happy', 'Sad', 'Victorious', 'Happy',
       'Victorious', 'Sad', 'Victorious', 'Happy', 'Victorious',
       'Victorious', 'Sad', 'Happy', 'Happy', 'Happy', 'Victorious',
       'Sad', 'Happy', 'Sad', 'Happy', 'Victorious', 'Sad', 'Victorious',
       'Sad', 'Happy', 'Happy', 'Sad', 'Happy', 'Victorious',
       'Victorious', 'Victorious', 'Victorious', 'Victorious', 'Sad',
       'Sad', 'Victorious', 'Victorious', 'Happy', 'Happy', 'Sad',
       'Happy', 'Sad', 'Happy', 'Victorious', 'Sad', 'Happy', 'Happy',
       'Sad', 'Sad', 'Victorious', 'Victorious', 'Sad', 'Sad',
       'Victorious', 'Victorious', 'Happy', 'Sad', 'Sad', 'Sad', 'Happy',
       'Sad', 'Victorious', 'Happy', 'Happy', 'Happy', 'Happy',
       'Victorious', 'Happy', 'Victorious', 'Vi

In [121]:
y_test

393           Sad
594    Victorious
363           Sad
446    Victorious
300           Sad
          ...    
323           Sad
588    Victorious
386           Sad
600    Victorious
542    Victorious
Name: class, Length: 192, dtype: object

In [122]:
# Saving our best model random forest classifier
with open('body_language.pkl', 'wb') as f:
    pickle.dump(fit_models['rf'],f)

4. Make Detections with Model

In [13]:
# fun part reloading our model into notebook
with open('body_language.pkl', 'rb') as f:
    model = pickle.load(f)

In [16]:
 model.predict(X_test)

NameError: name 'X_test' is not defined

In [18]:
# workflow for Detections
# 1. Detect landmarks
# 2. Predict pose(aka class) based on landmark coords
# 3. Render Landmarks and body language pose using OpenCV
cap = cv2.VideoCapture(1)
# Initiate holistic model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    while cap.isOpened():
        ret, frame = cap.read()
# When we read our image through open-cv but default the feed we get back is in BGR format
# But in order to work with media pipe we need our frames to be in RGB format
# So we recolor our feed        
        # Recolor Feed
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    # For peformance improvement we set writable flags to false
    # to prevent copying the image data but you're also then able to use the same image for rendering
        image.flags.writeable = False        
    # finally we use holistic model form the with line, using the process method to actually make detections
    # in which we pass our image variable    
        # Make Detections
        results = holistic.process(image)
        # print(results.face_landmarks)
        
        # face_landmarks, pose_landmarks, left_hand_landmarks, right_hand_landmarks
        
     # flatten() collapses an array into single method   # Recolor image back to BGR for rendering
        image.flags.writeable = True   
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # 1. Draw face landmarks
        mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS, 
                                 mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
                                 mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                                 )
        
        # 2. Right hand
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                                 )

        # 3. Left Hand
        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                                 )

        # 4. Pose Detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                 )
        # Export coordinates
        try:
            # Extract Pose landmarks
            pose = results.pose_landmarks.landmark
            pose_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in pose]).flatten())
            # flatten() collapses an array into single method
            # Extract Face landmarks
            face = results.face_landmarks.landmark
            face_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in face]).flatten())
            
            # Concate rows
            row = pose_row+face_row
            
#             # Append class name 
#             row.insert(0, class_name)
#             # beacause len(row) should be equal to 1+501*4
            
#             # Export to CSV
#             with open('coords.csv', mode='a', newline='') as f:
#                 csv_writer = csv.writer(f, delimiter=',', quotechar='"',quoting=csv.QUOTE_MINIMAL)
#                 csv_writer.writerow(row) 
            
        # MAKE DETECTION
            X = pd.DataFrame([row])
            body_language_class = model.predict(X)[0]
            body_language_prob = model.predict_proba(X)[0]
            print(body_language_class, body_language_prob)
            
        # Getting right ear coordinates
            coords = tuple(np.multiply(np.array((results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].x, results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].y)), [640,480]).astype(int))
            cv2.rectangle(image, 
                          (coords[0], coords[1]+5), 
                          (coords[0]+len(body_language_class)*20, coords[1]-30), 
                          (245, 117, 16), -1)
            cv2.putText(image, body_language_class, coords, 
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            
            # Get status box
            cv2.rectangle(image, (0,0), (250, 60), (245, 117, 16), -1)
            
            # Display Class
            cv2.putText(image, 'CLASS'
                        , (95,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
            cv2.putText(image, body_language_class.split(' ')[0]
                        , (90,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            
            # Display Probability
            cv2.putText(image, 'PROB'
                        , (15,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
            cv2.putText(image, str(round(body_language_prob[np.argmax(body_language_prob)],2))
                        , (10,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        except:
            pass
                        
        cv2.imshow('Raw Webcam Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()